In [1]:
import os
import sys
import numpy as np

from railway import *

import plotly.graph_objects as go
from plotly.offline import plot

# Root folder directory
ROOT_DIR = "/home/marco/railway-scheduling"

# Set the current directory to root directory
os.chdir(ROOT_DIR)
sys.path.append(os.getcwd())
print(f"Current working directory: 📂 {os.getcwd()}")

# Uselful functions
def darken_color(hex_color, percent=60):
    """Make a hex color darker by specified percentage"""
    # Remove '#' if present
    hex_color = hex_color.lstrip('#')
    
    # Convert hex to RGB
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    
    # Calculate darker RGB (multiply by factor)
    factor = 1 - (percent / 100)
    r = max(0, int(r * factor))
    g = max(0, int(g * factor))
    b = max(0, int(b * factor))
    
    # Convert back to hex
    return f'#{r:02x}{g:02x}{b:02x}'

Current working directory: 📂 /home/marco/railway-scheduling


In [2]:
# Constant values
n = 5
periods = 10
jobs = 10
passengers = 2000
K = 3

# Sets
Aj = {
    1: [(1, 5)],
    2: [(3, 5), (1, 3), (1, 2)],
    3: [(2, 3), (3, 4)],
    4: [(2, 3)],
    5: [(2, 3), (3, 4), (2, 5), (2,4)],
    6: [(1, 2)],
    7: [(4, 5), (1, 5), (2, 4)],
    8: [(2, 3), (2, 4)],
    9: [(1, 4)],
    10: [(1, 4), (1, 3)],
}

pi = {
    1: 3,
    2: 2,
    3: 1,
    4: 1,
    5: 1,
    6: 4,
    7: 1,
    8: 2,
    9: 1,
    10: 1
}

tau = {
    (1, 2): 0,
    (1, 3): 1,
    (1, 4): 3,
    (1, 5): 0,
    (2, 3): 1,
    (2, 4): 0,
    (2, 5): 0,
    (3, 4): 2,
    (3, 5): 1,
    (4, 5): 0,
}

R = {
 (1, 2, 1): [(1, 2)],
 (1, 2, 2): [(1, 3), (2, 3)],
 (1, 2, 3): [(1, 5), (2, 5)],
 (1, 3, 1): [(1, 3)],
 (1, 3, 2): [(1, 2), (2, 3)],
 (1, 3, 3): [(1, 5), (3, 5)],
 (1, 4, 1): [(1, 4)],
 (1, 4, 2): [(1, 5), (4, 5)],
 (1, 4, 3): [(1, 3), (3, 4)],
 (1, 5, 1): [(1, 5)],
 (1, 5, 2): [(1, 2), (2, 5)],
 (1, 5, 3): [(1, 4), (4, 5)],
 (2, 1, 1): [(1, 2)],
 (2, 1, 2): [(2, 3), (1, 3)],
 (2, 1, 3): [(2, 5), (1, 5)],
 (2, 3, 1): [(2, 3)],
 (2, 3, 2): [(1, 2), (1, 3)],
 (2, 3, 3): [(2, 4), (3, 4)],
 (2, 4, 1): [(2, 4)],
 (2, 4, 2): [(2, 3), (3, 4)],
 (2, 4, 3): [(2, 5), (4, 5)],
 (2, 5, 1): [(2, 5)],
 (2, 5, 2): [(1, 2), (1, 5)],
 (2, 5, 3): [(2, 3), (3, 5)],
 (3, 1, 1): [(1, 3)],
 (3, 1, 2): [(2, 3), (1, 2)],
 (3, 1, 3): [(3, 5), (1, 5)],
 (3, 2, 1): [(2, 3)],
 (3, 2, 2): [(1, 3), (1, 2)],
 (3, 2, 3): [(3, 4), (2, 4)],
 (3, 4, 1): [(3, 4)],
 (3, 4, 2): [(2, 3), (2, 4)],
 (3, 4, 3): [(3, 5), (4, 5)],
 (3, 5, 1): [(3, 5)],
 (3, 5, 2): [(3, 4), (4, 5)],
 (3, 5, 3): [(2, 3), (2, 5)],
 (4, 1, 1): [(1, 4)],
 (4, 1, 2): [(4, 5), (1, 5)],
 (4, 1, 3): [(3, 4), (1, 3)],
 (4, 2, 1): [(2, 4)],
 (4, 2, 2): [(3, 4), (2, 3)],
 (4, 2, 3): [(4, 5), (2, 5)],
 (4, 3, 1): [(3, 4)],
 (4, 3, 2): [(2, 4), (2, 3)],
 (4, 3, 3): [(4, 5), (3, 5)],
 (4, 5, 1): [(4, 5)],
 (4, 5, 2): [(3, 4), (3, 5)],
 (4, 5, 3): [(1, 4), (1, 5)],
 (5, 1, 1): [(1, 5)],
 (5, 1, 2): [(2, 5), (1, 2)],
 (5, 1, 3): [(4, 5), (1, 4)],
 (5, 2, 1): [(2, 5)],
 (5, 2, 2): [(1, 5), (1, 2)],
 (5, 2, 3): [(3, 5), (2, 3)],
 (5, 3, 1): [(3, 5)],
 (5, 3, 2): [(4, 5), (3, 4)],
 (5, 3, 3): [(2, 5), (2, 3)],
 (5, 4, 1): [(4, 5)],
 (5, 4, 2): [(3, 5), (3, 4)],
 (5, 4, 3): [(1, 5), (1, 4)]
}

C = [
    ((4, 5), (3, 5)),
    ((1, 4), (2, 5), (1, 2)),
    # ((1, 4), (2, 5)),
]

E = {
    # 1: {(5, 4): [(2, 5), (2, 4)]},
    # 2: {},
    # 3: {},
    # 4: {(4, 4): [(4, 5), (3, 5), (3, 4)]},
    # 5: {(3, 3): [(3, 5), (1, 5), (1, 3), (2, 3)]},
    # 6: {},
    # 7: {},
    # 8: {(5, 4): [(2, 5), (2, 4)]},
    # 9: {},
    # 10: {},
}

phi = {
    (1, 2, 1): 1243,
    (1, 2, 2): 832,
    (1, 2, 3): 918,
    (1, 2, 4): 143,
    (1, 2, 5): 767,
    (1, 2, 6): 1268,
    (1, 2, 7): 1587,
    (1, 2, 8): 1987,
    (1, 2, 9): 996,
    (1, 2, 10): 1803,
    (1, 3, 1): 1897,
    (1, 3, 2): 688,
    (1, 3, 3): 1819,
    (1, 3, 4): 1853,
    (1, 3, 5): 143,
    (1, 3, 6): 1027,
    (1, 3, 7): 261,
    (1, 3, 8): 370,
    (1, 3, 9): 194,
    (1, 3, 10): 774,
    (1, 4, 1): 373,
    (1, 4, 2): 1869,
    (1, 4, 3): 226,
    (1, 4, 4): 294,
    (1, 4, 5): 1650,
    (1, 4, 6): 1403,
    (1, 4, 7): 739,
    (1, 4, 8): 376,
    (1, 4, 9): 354,
    (1, 4, 10): 1588,
    (1, 5, 1): 1136,
    (1, 5, 2): 582,
    (1, 5, 3): 1537,
    (1, 5, 4): 1637,
    (1, 5, 5): 13,
    (1, 5, 6): 962,
    (1, 5, 7): 1728,
    (1, 5, 8): 1807,
    (1, 5, 9): 168,
    (1, 5, 10): 1192,
    (2, 1, 1): 64,
    (2, 1, 2): 1541,
    (2, 1, 3): 443,
    (2, 1, 4): 1176,
    (2, 1, 5): 1648,
    (2, 1, 6): 607,
    (2, 1, 7): 465,
    (2, 1, 8): 1315,
    (2, 1, 9): 250,
    (2, 1, 10): 1630,
    (2, 3, 1): 1683,
    (2, 3, 2): 112,
    (2, 3, 3): 1373,
    (2, 3, 4): 471,
    (2, 3, 5): 291,
    (2, 3, 6): 998,
    (2, 3, 7): 706,
    (2, 3, 8): 1261,
    (2, 3, 9): 1601,
    (2, 3, 10): 1136,
    (2, 4, 1): 1633,
    (2, 4, 2): 256,
    (2, 4, 3): 1923,
    (2, 4, 4): 331,
    (2, 4, 5): 630,
    (2, 4, 6): 473,
    (2, 4, 7): 1191,
    (2, 4, 8): 1735,
    (2, 4, 9): 166,
    (2, 4, 10): 634,
    (2, 5, 1): 447,
    (2, 5, 2): 173,
    (2, 5, 3): 149,
    (2, 5, 4): 1703,
    (2, 5, 5): 1646,
    (2, 5, 6): 1407,
    (2, 5, 7): 486,
    (2, 5, 8): 1341,
    (2, 5, 9): 342,
    (2, 5, 10): 1430,
    (3, 1, 1): 1438,
    (3, 1, 2): 561,
    (3, 1, 3): 735,
    (3, 1, 4): 1214,
    (3, 1, 5): 1065,
    (3, 1, 6): 416,
    (3, 1, 7): 1366,
    (3, 1, 8): 936,
    (3, 1, 9): 877,
    (3, 1, 10): 280,
    (3, 2, 1): 713,
    (3, 2, 2): 1136,
    (3, 2, 3): 142,
    (3, 2, 4): 137,
    (3, 2, 5): 636,
    (3, 2, 6): 266,
    (3, 2, 7): 1971,
    (3, 2, 8): 475,
    (3, 2, 9): 1704,
    (3, 2, 10): 1302,
    (3, 4, 1): 1168,
    (3, 4, 2): 658,
    (3, 4, 3): 473,
    (3, 4, 4): 256,
    (3, 4, 5): 1506,
    (3, 4, 6): 1074,
    (3, 4, 7): 867,
    (3, 4, 8): 3,
    (3, 4, 9): 1001,
    (3, 4, 10): 798,
    (3, 5, 1): 1120,
    (3, 5, 2): 1868,
    (3, 5, 3): 762,
    (3, 5, 4): 214,
    (3, 5, 5): 1434,
    (3, 5, 6): 1772,
    (3, 5, 7): 642,
    (3, 5, 8): 779,
    (3, 5, 9): 315,
    (3, 5, 10): 1968,
    (4, 1, 1): 387,
    (4, 1, 2): 1376,
    (4, 1, 3): 1789,
    (4, 1, 4): 1186,
    (4, 1, 5): 1476,
    (4, 1, 6): 483,
    (4, 1, 7): 326,
    (4, 1, 8): 1956,
    (4, 1, 9): 945,
    (4, 1, 10): 713,
    (4, 2, 1): 102,
    (4, 2, 2): 251,
    (4, 2, 3): 1954,
    (4, 2, 4): 672,
    (4, 2, 5): 1983,
    (4, 2, 6): 1660,
    (4, 2, 7): 1043,
    (4, 2, 8): 1098,
    (4, 2, 9): 1460,
    (4, 2, 10): 1983,
    (4, 3, 1): 665,
    (4, 3, 2): 1336,
    (4, 3, 3): 1051,
    (4, 3, 4): 1434,
    (4, 3, 5): 993,
    (4, 3, 6): 1768,
    (4, 3, 7): 1641,
    (4, 3, 8): 1912,
    (4, 3, 9): 1227,
    (4, 3, 10): 769,
    (4, 5, 1): 1639,
    (4, 5, 2): 447,
    (4, 5, 3): 967,
    (4, 5, 4): 1508,
    (4, 5, 5): 695,
    (4, 5, 6): 351,
    (4, 5, 7): 442,
    (4, 5, 8): 420,
    (4, 5, 9): 633,
    (4, 5, 10): 822,
    (5, 1, 1): 1078,
    (5, 1, 2): 738,
    (5, 1, 3): 381,
    (5, 1, 4): 1312,
    (5, 1, 5): 1615,
    (5, 1, 6): 1172,
    (5, 1, 7): 1537,
    (5, 1, 8): 1347,
    (5, 1, 9): 409,
    (5, 1, 10): 579,
    (5, 2, 1): 106,
    (5, 2, 2): 1017,
    (5, 2, 3): 1873,
    (5, 2, 4): 526,
    (5, 2, 5): 989,
    (5, 2, 6): 993,
    (5, 2, 7): 1012,
    (5, 2, 8): 116,
    (5, 2, 9): 753,
    (5, 2, 10): 1698,
    (5, 3, 1): 95,
    (5, 3, 2): 207,
    (5, 3, 3): 565,
    (5, 3, 4): 1370,
    (5, 3, 5): 1167,
    (5, 3, 6): 1656,
    (5, 3, 7): 1950,
    (5, 3, 8): 1713,
    (5, 3, 9): 1906,
    (5, 3, 10): 1569,
    (5, 4, 1): 348,
    (5, 4, 2): 1974,
    (5, 4, 3): 1194,
    (5, 4, 4): 1703,
    (5, 4, 5): 742,
    (5, 4, 6): 771,
    (5, 4, 7): 1017,
    (5, 4, 8): 1208,
    (5, 4, 9): 1166,
    (5, 4, 10): 243,
}

beta = {
    (1, 2, 1): 0.2708875131672326,
    (1, 2, 2): 0.029612432141835288,
    (1, 2, 3): 0.47962781252859066,
    (1, 2, 4): 0.7455633078004509,
    (1, 2, 5): 0.0214517122004797,
    (1, 2, 6): 0.2578945475570307,
    (1, 2, 7): 0.07396337576917633,
    (1, 2, 8): 0.3276453624875285,
    (1, 2, 9): 0.0034525277219513706,
    (1, 2, 10): 0.6039945615130173,
    (1, 3, 1): 0.07474475058731656,
    (1, 3, 2): 0.4075760502388067,
    (1, 3, 3): 0.9784121662625731,
    (1, 3, 4): 0.09755745857098386,
    (1, 3, 5): 0.21733874941523912,
    (1, 3, 6): 0.5134505657311303,
    (1, 3, 7): 0.029695517549054196,
    (1, 3, 8): 0.06171803515993657,
    (1, 3, 9): 0.48062199046691734,
    (1, 3, 10): 0.7935246688973094,
    (1, 4, 1): 0.716435182125516,
    (1, 4, 2): 0.7006725880048127,
    (1, 4, 3): 0.3428500663434344,
    (1, 4, 4): 0.07827016363185979,
    (1, 4, 5): 0.804355366965485,
    (1, 4, 6): 0.6820687742731577,
    (1, 4, 7): 0.7008774484209404,
    (1, 4, 8): 0.7224909424114986,
    (1, 4, 9): 0.5909780739970357,
    (1, 4, 10): 0.33674753132279356,
    (1, 5, 1): 0.7995917364884482,
    (1, 5, 2): 0.9485613276977536,
    (1, 5, 3): 0.8490893631615071,
    (1, 5, 4): 0.05559245660009615,
    (1, 5, 5): 0.4299897739268913,
    (1, 5, 6): 0.6520655340693398,
    (1, 5, 7): 0.32419370637315426,
    (1, 5, 8): 0.5485721903711065,
    (1, 5, 9): 0.6049448164341475,
    (1, 5, 10): 0.962983652533092,
    (2, 1, 1): 0.07216642302319842,
    (2, 1, 2): 0.1583707616491361,
    (2, 1, 3): 0.2997323984595426,
    (2, 1, 4): 0.8506133972492431,
    (2, 1, 5): 0.43728160408189887,
    (2, 1, 6): 0.809679612629752,
    (2, 1, 7): 0.39866372097734637,
    (2, 1, 8): 0.7639038098312094,
    (2, 1, 9): 0.08925651021390191,
    (2, 1, 10): 0.6943843495019008,
    (2, 3, 1): 0.7705328137400858,
    (2, 3, 2): 0.7481461189009233,
    (2, 3, 3): 0.4632904166521118,
    (2, 3, 4): 0.794143537395621,
    (2, 3, 5): 0.16650032687351635,
    (2, 3, 6): 0.6835560769511594,
    (2, 3, 7): 0.141685255864211,
    (2, 3, 8): 0.9323532786683408,
    (2, 3, 9): 0.05405644489974881,
    (2, 3, 10): 0.5296192762098444,
    (2, 4, 1): 0.4754403032149963,
    (2, 4, 2): 0.8051744717876043,
    (2, 4, 3): 0.9248330387344341,
    (2, 4, 4): 0.3418071742453339,
    (2, 4, 5): 0.6278415222965852,
    (2, 4, 6): 0.6205060896372127,
    (2, 4, 7): 0.8277337516644012,
    (2, 4, 8): 0.1500074680662714,
    (2, 4, 9): 0.18754676959361194,
    (2, 4, 10): 0.9568624805926145,
    (2, 5, 1): 0.3679723448224138,
    (2, 5, 2): 0.6881989686629635,
    (2, 5, 3): 0.31912109820117407,
    (2, 5, 4): 0.04330618994739488,
    (2, 5, 5): 0.6212627809019029,
    (2, 5, 6): 0.5917539344474513,
    (2, 5, 7): 0.6192048677510374,
    (2, 5, 8): 0.11483329027671818,
    (2, 5, 9): 0.1924915850505885,
    (2, 5, 10): 0.4938502434916987,
    (3, 1, 1): 0.7156713337868706,
    (3, 1, 2): 0.9899160266437795,
    (3, 1, 3): 0.21879914664562827,
    (3, 1, 4): 0.8702864947472593,
    (3, 1, 5): 0.7420942132777929,
    (3, 1, 6): 0.181777284193401,
    (3, 1, 7): 0.7895381871423821,
    (3, 1, 8): 0.31848662618742407,
    (3, 1, 9): 0.5120798892052265,
    (3, 1, 10): 0.6791216284234978,
    (3, 2, 1): 0.6160214255794815,
    (3, 2, 2): 0.6667937677735374,
    (3, 2, 3): 0.8224802569092093,
    (3, 2, 4): 0.25617357867418766,
    (3, 2, 5): 0.9532446666313605,
    (3, 2, 6): 0.9491979189338181,
    (3, 2, 7): 0.1616625788077366,
    (3, 2, 8): 0.10592288772518743,
    (3, 2, 9): 0.7659962075460004,
    (3, 2, 10): 0.2617390604080255,
    (3, 4, 1): 0.9540940683963051,
    (3, 4, 2): 0.7071622990897389,
    (3, 4, 3): 0.094660223355996,
    (3, 4, 4): 0.2447267434686441,
    (3, 4, 5): 0.3151372712657593,
    (3, 4, 6): 0.3059796995736197,
    (3, 4, 7): 0.7759434961696052,
    (3, 4, 8): 0.941167596858192,
    (3, 4, 9): 0.1549061597596073,
    (3, 4, 10): 0.021420387348824788,
    (3, 5, 1): 0.6005122973850641,
    (3, 5, 2): 0.17629260516633283,
    (3, 5, 3): 0.8209591097628683,
    (3, 5, 4): 0.49936167659235375,
    (3, 5, 5): 0.8883323022633904,
    (3, 5, 6): 0.2095669561912138,
    (3, 5, 7): 0.35143646946805784,
    (3, 5, 8): 0.2579970787861242,
    (3, 5, 9): 0.00814166673555794,
    (3, 5, 10): 0.2524779250403817,
    (4, 1, 1): 0.9857305246780448,
    (4, 1, 2): 0.9486833396784156,
    (4, 1, 3): 0.0557743838247543,
    (4, 1, 4): 0.6815436431701414,
    (4, 1, 5): 0.707579934694827,
    (4, 1, 6): 0.8325639734693631,
    (4, 1, 7): 0.4696112349980949,
    (4, 1, 8): 0.6189823726447907,
    (4, 1, 9): 0.6899445477980266,
    (4, 1, 10): 0.6201096884571816,
    (4, 2, 1): 0.9893264471189575,
    (4, 2, 2): 0.7905170365236918,
    (4, 2, 3): 0.6419025271855879,
    (4, 2, 4): 0.3932966676772315,
    (4, 2, 5): 0.08094685496738618,
    (4, 2, 6): 0.3054792178758047,
    (4, 2, 7): 0.6036138825627252,
    (4, 2, 8): 0.8298875104725527,
    (4, 2, 9): 0.016672456802417246,
    (4, 2, 10): 0.0999200850425861,
    (4, 3, 1): 0.41258478629265605,
    (4, 3, 2): 0.38470990045732434,
    (4, 3, 3): 0.5603851779889351,
    (4, 3, 4): 0.3617820471187604,
    (4, 3, 5): 0.6636231016563378,
    (4, 3, 6): 0.2591352878251286,
    (4, 3, 7): 0.48849861127916494,
    (4, 3, 8): 0.8752323746393157,
    (4, 3, 9): 0.8461234483358656,
    (4, 3, 10): 0.09220717352482799,
    (4, 5, 1): 0.1708100581109726,
    (4, 5, 2): 0.2936230028165394,
    (4, 5, 3): 0.012799419611955742,
    (4, 5, 4): 0.5681176847495927,
    (4, 5, 5): 0.48586782751389845,
    (4, 5, 6): 0.6520925227475753,
    (4, 5, 7): 0.8010660305438153,
    (4, 5, 8): 0.4939509567179877,
    (4, 5, 9): 0.34987525914308415,
    (4, 5, 10): 0.8499243897050031,
    (5, 1, 1): 0.074192486177042,
    (5, 1, 2): 0.09843574144426503,
    (5, 1, 3): 0.6816977577954125,
    (5, 1, 4): 0.7808873075420224,
    (5, 1, 5): 0.18675907263806835,
    (5, 1, 6): 0.7638483987719894,
    (5, 1, 7): 0.26064497714111023,
    (5, 1, 8): 0.820080006615796,
    (5, 1, 9): 0.8430069128827619,
    (5, 1, 10): 0.5354049554274177,
    (5, 2, 1): 0.1847234010124531,
    (5, 2, 2): 0.4948197359214921,
    (5, 2, 3): 0.930512645954611,
    (5, 2, 4): 0.2846119072092168,
    (5, 2, 5): 0.8177829777275863,
    (5, 2, 6): 0.8933582600170231,
    (5, 2, 7): 0.7856191201117175,
    (5, 2, 8): 0.982899113681734,
    (5, 2, 9): 0.5201811551537777,
    (5, 2, 10): 0.10871673482221988,
    (5, 3, 1): 0.08348379906433445,
    (5, 3, 2): 0.6915071022822251,
    (5, 3, 3): 0.6464654688739807,
    (5, 3, 4): 0.7555486227242346,
    (5, 3, 5): 0.9446696449935775,
    (5, 3, 6): 0.2928793877975677,
    (5, 3, 7): 0.03783405222557612,
    (5, 3, 8): 0.10933169124492692,
    (5, 3, 9): 0.6924104848619352,
    (5, 3, 10): 0.560100324381403,
    (5, 4, 1): 0.9077639375805606,
    (5, 4, 2): 0.5604650389097575,
    (5, 4, 3): 0.854942435738218,
    (5, 4, 4): 0.6820362375969844,
    (5, 4, 5): 0.3757180657336774,
    (5, 4, 6): 0.06570308708399808,
    (5, 4, 7): 0.15313132074699765,
    (5, 4, 8): 0.6766312196981439,
    (5, 4, 9): 0.2638998444815207,
    (5, 4, 10): 0.46907153627818454,
}

Lambd = {
    ((1, 2), 1): 144,
    ((1, 2), 2): 137,
    ((1, 2), 3): 169,
    ((1, 2), 4): 213,
    ((1, 2), 5): 121,
    ((1, 2), 6): 189,
    ((1, 2), 7): 172,
    ((1, 2), 8): 270,
    ((1, 2), 9): 205,
    ((1, 2), 10): 132,
    ((1, 3), 1): 242,
    ((1, 3), 2): 207,
    ((1, 3), 3): 269,
    ((1, 3), 4): 220,
    ((1, 3), 5): 269,
    ((1, 3), 6): 265,
    ((1, 3), 7): 190,
    ((1, 3), 8): 178,
    ((1, 3), 9): 135,
    ((1, 3), 10): 204,
    ((1, 4), 1): 200,
    ((1, 4), 2): 181,
    ((1, 4), 3): 172,
    ((1, 4), 4): 257,
    ((1, 4), 5): 219,
    ((1, 4), 6): 223,
    ((1, 4), 7): 161,
    ((1, 4), 8): 204,
    ((1, 4), 9): 159,
    ((1, 4), 10): 132,
    ((1, 5), 1): 203,
    ((1, 5), 2): 158,
    ((1, 5), 3): 184,
    ((1, 5), 4): 267,
    ((1, 5), 5): 201,
    ((1, 5), 6): 210,
    ((1, 5), 7): 275,
    ((1, 5), 8): 159,
    ((1, 5), 9): 183,
    ((1, 5), 10): 235,
    ((2, 3), 1): 242,
    ((2, 3), 2): 216,
    ((2, 3), 3): 264,
    ((2, 3), 4): 176,
    ((2, 3), 5): 268,
    ((2, 3), 6): 209,
    ((2, 3), 7): 249,
    ((2, 3), 8): 240,
    ((2, 3), 9): 212,
    ((2, 3), 10): 268,
    ((2, 4), 1): 269,
    ((2, 4), 2): 124,
    ((2, 4), 3): 127,
    ((2, 4), 4): 258,
    ((2, 4), 5): 145,
    ((2, 4), 6): 160,
    ((2, 4), 7): 133,
    ((2, 4), 8): 144,
    ((2, 4), 9): 253,
    ((2, 4), 10): 242,
    ((2, 5), 1): 214,
    ((2, 5), 2): 247,
    ((2, 5), 3): 124,
    ((2, 5), 4): 242,
    ((2, 5), 5): 234,
    ((2, 5), 6): 228,
    ((2, 5), 7): 211,
    ((2, 5), 8): 187,
    ((2, 5), 9): 254,
    ((2, 5), 10): 222,
    ((3, 4), 1): 254,
    ((3, 4), 2): 279,
    ((3, 4), 3): 243,
    ((3, 4), 4): 177,
    ((3, 4), 5): 217,
    ((3, 4), 6): 224,
    ((3, 4), 7): 275,
    ((3, 4), 8): 237,
    ((3, 4), 9): 176,
    ((3, 4), 10): 189,
    ((3, 5), 1): 157,
    ((3, 5), 2): 202,
    ((3, 5), 3): 277,
    ((3, 5), 4): 130,
    ((3, 5), 5): 254,
    ((3, 5), 6): 207,
    ((3, 5), 7): 277,
    ((3, 5), 8): 253,
    ((3, 5), 9): 139,
    ((3, 5), 10): 195,
    ((4, 5), 1): 206,
    ((4, 5), 2): 245,
    ((4, 5), 3): 152,
    ((4, 5), 4): 235,
    ((4, 5), 5): 175,
    ((4, 5), 6): 132,
    ((4, 5), 7): 238,
    ((4, 5), 8): 136,
    ((4, 5), 9): 264,
    ((4, 5), 10): 244,
}

coords = [
    (0.3044084064007771, 0.6762812025741588),
    (-0.4721935591312115, 0.682051096593445),
    (-0.65766344679408496, 0.05428002261931836),
    (-0.2508324145918874, -0.6568744615640193),
    (0.49290862504363805, -0.3218446977786867),
]

# Initialization
rail = Railway(
    stations=n,
    periods=periods,
    jobs=jobs,
    passengers=passengers,
    routes=K,
    coords=coords,
    pi=pi,
    Aj=Aj,
    tau=tau,
    phi=phi,
    beta=beta,
    Lambd=Lambd,
    R=R,
    C=C,
    E=E,    
)

# Generate missing values
# rail._Railway__generate_R()
# rail._Railway__generate_phi(min_demand=0, max_demand=1)
# rail._Railway__generate_beta(min_share=0, max_share=1)
# rail._Railway__generate_Lambd(min_capacity=0.3, max_capacity=0.7)
# rail._Railway__generate_E(n_max_events=1, min_length=1, max_length=3)

Set parameter Username
Set parameter LicenseID to value 2629256
Academic license - for non-commercial use only - expires 2026-02-27


In [3]:
# Variables for plotting
N = rail.N
coords = rail.coords
coords_plot = []
for (x, y), i in zip(coords, N):
	coords_plot.append((x, y, i))
A = rail.A
Ja = rail.Ja
omega_e = rail.omega_e
omega_j = rail.omega_j
J = rail.J
Aj = rail.Aj

buttons = []
traces = {}

# Plot
fig = go.Figure()

# Gray background rectangle
fig.add_shape(
    type="rect",
    x0=-1.1, y0=-1.1, x1=1.1, y1=1.1,
    fillcolor="lightgray",
    line=dict(color="lightgray", width=0),
    layer="below"
)

# White circle
fig.add_shape(
    type="circle",
    x0=-1, y0=-1, x1=1, y1=1,
    fillcolor="white",
    line=dict(color="white", width=0),
    layer="below"
)

# Add dashed unit circle
theta = np.linspace(0, 2 * np.pi, 100)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
fig.add_trace(
    go.Scatter(
        x=circle_x, y=circle_y,
        mode="lines",
        line=dict(color="black", dash="dash", width=1),
        hoverinfo="skip",
        showlegend=False
    )
)

# Add arcs (gray lines between stations)
for i, j in A:
    i_idx = i - 1  # Adjust for 0-based indexing
    j_idx = j - 1
    x0, y0 = coords[i_idx]
    x1, y1 = coords[j_idx]
        
    # Arc lines
    fig.add_trace(
        go.Scatter(
            x=[x0, x1], y=[y0, y1],
            mode="lines",
            line=dict(color="gray", width=3),
            showlegend=False
        )
    )
    
    # Arcs invisible hover markers
    mid_x = (x0 + x1) / 2
    mid_y = (y0 + y1) / 2
    fig.add_trace(
        go.Scatter(
            x=[mid_x], y=[mid_y],
            mode="markers",
            marker=dict(color="lightgray", size=0, opacity=0),
            hovertext=fr"Arc: ({i}, {j})<br>ω_e: {omega_e[(i, j)]:.2f}<br>ω_j: {omega_j[(i, j)]:.2f}<br>Jobs: " + f"{Ja[(i, j)]}" if (i, j) in Ja else "None",
            hoverinfo="text",
            showlegend=False
        )
    )

# Add colored arcs for jobs (if provided)
if J is not None and Aj is not None:

    # Build a colorscale for the jobs
    import plotly.express as px
    light24 = px.colors.qualitative.Light24
    light24[12] = '#C4451C' # substitute gray color

    # Loop over jobs in the range rail.J 
    colorscale = []
    for i in rail.J:
        color = light24[(i - 1) % len(light24)]
        colorscale.append(color)   
    
    # Create a list to store trace indices for buttons
    job_trace_indices = {}

    # for j_idx, (j, color) in enumerate(zip(J, colorscale)):
    #     if j in Aj:
    #         arc_coords_j = []
    #         for a in Aj[j]:
    #             # Find arc coordinates
    #             a_origin = coords[a[0] - 1]
    #             arc_coords_j.append(a_origin) 
    #             a_destination = coords[a[1] - 1]
    #             arc_coords_j.append(a_destination)

    #         print(j, arc_coords_j)

    #         # Plot the arc for the job
    #         trace = go.Scatter(
    #             x=[coord[0] for coord in arc_coords_j],
    #             y=[coord[1] for coord in arc_coords_j],
    #             mode="lines",
    #             visible=False,
    #             line=dict(color=color, width=3),
    #             name=f"Job {j}",
    #             hovertext=f"Job {j}<br>Arc: ({a[0]}, {a[1]})",
    #             hoverinfo="text",
    #             showlegend=False
    #         )
    #         traces[j] = trace
    #         fig.add_trace(trace)
    #         job_trace_indices[j] = len(fig.data) - 1  # Current index after adding

    
    
    # Replace the current job visualization code with this:
    for j_idx, (j, color) in enumerate(zip(J, colorscale)):
        if j in Aj:
            # Store trace indices for this job
            job_trace_indices[j] = []
            
            for a in Aj[j]:
                # Find arc coordinates (just get origin and destination directly)
                a_origin = coords[a[0] - 1]
                a_destination = coords[a[1] - 1]
                
                # Create a separate trace for each arc
                trace = go.Scatter(
                    x=[a_origin[0], a_destination[0]],
                    y=[a_origin[1], a_destination[1]],
                    mode="lines",
                    visible=False,
                    line=dict(color=color, width=3),
                    name=f"Job {j} - Arc {a}",
                    hovertext=f"Job {j}<br>Arc: ({a[0]}, {a[1]})",
                    hoverinfo="text",
                    showlegend=False
                )
                fig.add_trace(trace)
                job_trace_indices[j].append(len(fig.data) - 1)  # Add this trace index to the job's list 
    
    
    

# # Button for each job that TOGGLES just that job's visibility
# for j in J:
#     if j in job_trace_indices:
#         trace_idx = job_trace_indices[j]
#         button = dict(
#             method="restyle",  # Use restyle instead of update
#             label=f"Job {j}",
#             args=[{"visible": "toggle"}, [trace_idx]]  # Toggle only this trace
#         )
#         buttons.append(button)

# # "Show All" button sets all job traces to visible
# show_all_button = dict(
#     method="restyle",
#     label="All Jobs",
#     args=[{"visible": True}, [job_trace_indices[j] for j in job_trace_indices]]
# )

# # "Hide All" button hides all job traces
# hide_all_button = dict(
#     method="restyle",
#     label="Hide Jobs",
#     args=[{"visible": False}, [job_trace_indices[j] for j in job_trace_indices]]
# )



# Update the button creation code
for j in J:
    if j in job_trace_indices:
        trace_indices = job_trace_indices[j]
        button = dict(
            method="restyle",
            label=f"Job {j}",
            args=[{"visible": True}, trace_indices],  # Toggle all traces for this job
            args2=[{"visible": False}, trace_indices]  # Hide all traces for this job
        )
        buttons.append(button)

# Update show all/hide all buttons
all_job_trace_indices = [idx for j in job_trace_indices for idx in job_trace_indices[j]]

show_all_button = dict(
    method="restyle",
    label="All Jobs",
    args=[{"visible": True}, all_job_trace_indices]
)

hide_all_button = dict(
    method="restyle",
    label="Hide Jobs",
    args=[{"visible": False}, all_job_trace_indices]
)




# Add stations (blue dots)
fig.add_trace(go.Scatter(
    x=[x for x, _, _ in coords_plot],
    y=[y for _, y, _ in coords_plot],
    mode="markers",
    marker=dict(color="blue", size=15),
    text=[f"Station: {s}<br>X: {x:.2f}<br>Y: {y:.2f}" for x, y, s in coords_plot],
    hoverinfo="text",
    showlegend=False
))
# Add station numbers as text labels next to dots
fig.add_trace(go.Scatter(
    x=[x+3e-2 for x, _, _ in coords_plot],
    y=[y+3e-2 for _, y, _ in coords_plot],
    mode="text",
    text=[f"{s}" for _, _, s in coords_plot],
    textposition="top right",  # Position text to the top right of points
    textfont=dict(
        color="blue",
        size=20,
    ),
    hoverinfo="skip",  # Don't show additional hover info for these text elements
    showlegend=False
))

# Update layout
fig.update_layout(
    autosize=True,   # let the container size drive the plot
    # width=1000,
    # height=1000,
    margin=dict(l=10, r=10, t=10, b=10),
    xaxis=dict(
        range=[-1.1, 1.1],
        showgrid=False,
        zeroline=False,
        scaleanchor="y",
        scaleratio=1,
    ),
    yaxis=dict(
        range=[-1.1, 1.1],
        showgrid=False,
        zeroline=False,
        scaleanchor="x",
        scaleratio=1
    ),
    updatemenus=[
        dict(
            type='dropdown',
            buttons=[show_all_button, hide_all_button] + buttons,
            direction='down',
            showactive=True,
            x=0.98,
            xanchor='right',
            y=0.98,
            yanchor='top',
        ),
        # dict(
        #     type='buttons',
        #     buttons=[hide_all_button],
        #     direction='down',
        #     showactive=True,
        #     x=0.98,
        #     xanchor='right',
        #     y=0.93,
        #     yanchor='top',
        # )
    ],
    # legend=dict(
    #     yanchor="top",
    #     y=1,
    #     xanchor="left",
    #     x=1,
    # ),
    hovermode="closest",
    hoverlabel=dict(font_size=16),
)

fig.show()

# # Save as interactive html
# div_station_plot = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# # write div snippet to file
# with open('docs/images/stations.html', 'w') as f:
#     f.write(div_station_plot)


In [4]:
Aj

{1: [(1, 5)],
 2: [(3, 5), (1, 3), (1, 2)],
 3: [(2, 3), (3, 4)],
 4: [(2, 3)],
 5: [(2, 3), (3, 4), (2, 5), (2, 4)],
 6: [(1, 2)],
 7: [(4, 5), (1, 5), (2, 4)],
 8: [(2, 3), (2, 4)],
 9: [(1, 4)],
 10: [(1, 4), (1, 3)]}

In [5]:
# somma = (
#     sum(1 for a in c) <= 1
#     for t in rail.T
#     for c in rail.C
# )
# print(somma)
for c in rail.C:
    for a in c:
        print(*a)
        

4 5
3 5
1 4
2 5
1 2


In [6]:
# Set constraints
rail.set_constraints()

# Set objective
rail.set_objective()

# Silence output
# rail.model.setParam('OutputFlag', 0)
rail.model.setParam('Timelimit', 30)

# Most basic model
rail.model.setParam('LPWarmStart',0)
rail.model.setParam('PoolSolutions', 1)
rail.model.Params.presolve = 0
# rail.model.Params.MultiObjPre = 0
rail.model.Params.cuts = 0
rail.model.Params.cutpasses = 0
rail.model.Params.threads = 1
rail.model.Params.heuristics = 0
rail.model.Params.symmetry = 0

# Solve the scheduling problem
performance = rail.optimize()

Set parameter TimeLimit to value 30
Set parameter LPWarmStart to value 0
Set parameter PoolSolutions to value 1
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter CutPasses to value 0
Set parameter Threads to value 1
Set parameter Heuristics to value 0
Set parameter Symmetry to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Non-default parameters:
TimeLimit  30
LPWarmStart  0
Heuristics  0
Symmetry  0
Cuts  0
CutPasses  0
Presolve  0
Threads  1
PoolSolutions  1

Optimize a model with 2140 rows, 1100 columns and 6047 nonzeros
Model fingerprint: 0x188b48a8
Variable types: 300 continuous, 800 integer (800 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+05]
  Objective range  [3e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-01, 1e+0

In [7]:
# Set parameter TimeLimit to value 30
# Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

# CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
# Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

# Non-default parameters:
# TimeLimit  30

# Optimize a model with 2124 rows, 1100 columns and 6106 nonzeros
# Model fingerprint: 0xb25b9014
# Variable types: 300 continuous, 800 integer (800 binary)
# Coefficient statistics:
#   Matrix range     [2e-01, 1e+05]
#   Objective range  [3e+00, 2e+03]
#   Bounds range     [1e+00, 1e+00]
#   RHS range        [9e-01, 1e+05]
# Presolve removed 1695 rows and 821 columns
# Presolve time: 0.02s
# Presolved: 429 rows, 279 columns, 1919 nonzeros
# Variable types: 74 continuous, 205 integer (205 binary)
# Found heuristic solution: objective 18222.308920

# Root relaxation: objective 1.327962e+04, 425 iterations, 0.01 seconds (0.01 work units)

#     Nodes    |    Current Node    |     Objective Bounds      |     Work
#  Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

#      0     0 13279.6244    0  129 18222.3089 13279.6244  27.1%     -    0s
# H    0     0                    17507.734054 13279.6244  24.1%     -    0s
# H    0     0                    16868.490040 13279.6244  21.3%     -    0s
# H    0     0                    16123.956608 13279.6244  17.6%     -    0s
# H    0     0                    15290.719290 14842.0285  2.93%     -    0s
#      0     0 14842.0285    0   82 15290.7193 14842.0285  2.93%     -    0s
# H    0     0                    15268.626319 14873.7085  2.59%     -    0s
#      0     0 14906.2959    0   88 15268.6263 14906.2959  2.37%     -    0s
#      0     0 14910.5271    0   90 15268.6263 14910.5271  2.35%     -    0s
#      0     0 15132.1498    0   84 15268.6263 15132.1498  0.89%     -    0s
#      0     0     cutoff    0      15268.6263 15268.6263  0.00%     -    0s

# Explored 1 nodes (637 simplex iterations) in 0.20 seconds (0.05 work units)
# Thread count was 8 (of 8 available processors)

# Solution count 6: 15268.6 15290.7 16124 ... 18222.3

# Optimal solution found (tolerance 1.00e-04)
# Best objective 1.526862631871e+04, best bound 1.526862631871e+04, gap 0.0000%

In [8]:
# Preprocessing for plot

# Dictionary to associate names to arcs
arc_names = {f'a{i}{j}': (i, j) for i, j in rail.A}

# Dictionary to give each arc an order
arc_order = {arc: i for i, arc in enumerate(rail.A)}

In [9]:
fig = go.Figure()

# Preprocessing for plot

# Dictionary to associate names to arcs
arc_names = {f'a{i}{j}': (i, j) for i, j in rail.A}

# Dictionary to give each arc an order
arc_order = {arc: i for i, arc in enumerate(rail.A)}

# Horizontal white lines at every integer x value
for i in range(1,rail.Tend + 1):
    fig.add_shape(type="line",
        x0=i, y0=0, x1=i, y1=len(rail.A),
        line=dict(
            color='white',
            width=1,
        ),
    )

# Horizontal white lines at every integer y value
for i in range(len(rail.A)):
    fig.add_shape(type="line",
        x0=0, y0=i, x1=rail.Tend + 1, y1=i,
        line=dict(
            color='white',
            width=1,
        ),
    )

# Only for C set slide
# Background color for selected arc boxes
fig.add_shape(
    type="rect",
    x0=0,
    y0=arc_order[(4,5)],
    x1=rail.Tend + 1,
    y1=arc_order[(4,5)] + 1,
    fillcolor="red",
    opacity=0.4,
    line=dict(color="red", width=0),
    layer="below"
)
fig.add_shape(
    type="rect",
    x0=0,
    y0=arc_order[(3,5)],
    x1=rail.Tend + 1,
    y1=arc_order[(3,5)] + 1,
    fillcolor="red",
    opacity=0.4,
    line=dict(color="red", width=0),
    layer="below"
)

fig.add_shape(
    type="rect",
    x0=0,
    y0=arc_order[(1,4)],
    x1=rail.Tend + 1,
    y1=arc_order[(1,4)] + 1,
    fillcolor="purple",
    opacity=0.4,
    line=dict(color="purple", width=0),
    layer="below"
)
fig.add_shape(
    type="rect",
    x0=0,
    y0=arc_order[(2,5)],
    x1=rail.Tend + 1,
    y1=arc_order[(2,5)] + 1,
    fillcolor="purple",
    opacity=0.4,
    line=dict(color="purple", width=0),
    layer="below"
)
fig.add_shape(
    type="rect",
    x0=0,
    y0=arc_order[(1,2)],
    x1=rail.Tend + 1,
    y1=arc_order[(1,2)] + 1,
    fillcolor="purple",
    opacity=0.4,
    line=dict(color="purple", width=0),
    layer="below"
)

# Add shapes and hover points
jobs = list(rail.J)
num_arcs = len(rail.A)
x_label_positions = [i + 0.5 for i in range(1, rail.Tend + 1)]
y_label_positions = [i + 0.5 for i in range(len(rail.A))]
hover_x = []
hover_y = []
hover_text = []
hover_colors = []

# S = rail.generate_initial_solution()
# S = rail.generate_neighbor_solution(S0)
S = rail.S
Aj = rail.Aj

    
fig.update_shapes(dict(xref='x', yref='y'))

for j, color in zip(jobs, colorscale):
    for i_arc, arc in enumerate(rail.Aj[j]):
        fig.add_shape(type="rect",
            x0=S[j],
            y0=arc_order[Aj[j][i_arc]],
            x1=S[j] + rail.pi[j],
            y1=arc_order[Aj[j][i_arc]] + 1,
            line=dict(
                color=color,
                width=1,
            ),
            fillcolor=color,
        )

        x_center = S[j] + rail.pi[j] / 2
        y_center = arc_order[Aj[j][i_arc]] + 0.5
        hover_text = f'Job: {j}<br>Arc: {Aj[j][i_arc]}<br>Tau: {rail.tau[Aj[j][i_arc]]}<br>Start: {S[j]}<br>End: {S[j] + rail.pi[j]}'
        fig.add_annotation(
            x=x_center,
            y=y_center,
            text=f"Job {j}",
            showarrow=False,
            bgcolor=color,
            font=dict(
                color=darken_color(color),
                size=14,
            ),
            hovertext=hover_text,
        )
    
# Add black hollow boxes for events
for time, event in rail.E.items():
    for _, arcs in event.items():
        for arc in arcs:
            fig.add_shape(type="rect",
                x0=time,
                y0=arc_order[arc],
                x1=time + 1,
                y1=arc_order[arc] + 1,
                line=dict(
                    color='black',
                    width=2,
                ),
                fillcolor=None,
            )
            

# Layout
fig.update_layout(
    # width=800,
    # height=600,
    autosize=True,   # let the container size drive the plot
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis=dict(
        title="Time",
        range=[1, rail.Tend+1],
        tickvals=x_label_positions,
        ticktext=[f"{i}" for i in range(1, rail.Tend + 1)],
        showgrid=False,
    ),
    yaxis=dict(
        type="category",
        title="Arcs",
        range=[0, num_arcs],
        tickvals=y_label_positions,
        ticktext=list(arc_names.keys()),
        showgrid=False,
    ),
)

fig.show()

# # # Save as HTML
# div_schedule = plot(
#     fig, include_plotlyjs=True, config={"responsive": True}, output_type="div"
# )
# # write div snippet to file
# with open('docs/images/scheduleC2.html', 'w') as f:
#     f.write(div_schedule)


In [10]:

# S = rail.generate_initial_solution()
y, x, h, w, v = rail.get_vars_from_times(S)
f = rail.get_objective_value(v)
print(f)

print(rail.check_feasibility(y, x, h, w, v))

15228.234851335359
False
